# PERFORMANCE COMPRESOR DE LA TURBINA A GAS

**Descripción de los datos:** Los datos presentados son datos reales extraidos del historiador de una central termoeléctrica (Exacuantum). El proceso de extracción de datos consistió en identificar las variables de interés para el cálculo de la eficiencia del compresor de la turbina. El historiador tiene capacidad de tener registro de los datos desde 2018 y estos datos son almacenados segundo a segundo, por lo que para facilidad del modelo, se seleccionará fechas específicas sin embargo se puede trabajar con intervalos de tiempos más grandes para robustecer los modelos realizados.

In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import pandas as pd
import numpy as np

## 01. Carga, descripción y limpieza de datos


### Descripción de los Datos



DATE	:	Instante de registro de datos

G1_DWATT:	Potencia de la Turbina a Gas (TG11)

G2_DWATT:	Potencia de la Turbina a Gas (TG12)

S1_DWATT:	Potencia de la  Turbina a Vapor (TV10)

G1_CTIM	:	Temperatura Ingreso al Compresor TG11

G2_CTIM	:	Temperatura Ingreso al Compresor TG12

G1_CPR	:	Ratio de Compresión del TG11

G2_CPR	:	Ratio de Compresión del TG12

G1_CTD	:	Temperatura de Descarga del Compresor TG11

G2_CTD	:	Temperatura de Descarga del Compresor TG12

G1_CPD	:	Presiónde Descarga del Compresor TG11

G2_CPD	:	Presiónde Descarga del Compresor TG12

G1_AT1	:	Temperatura de Ingreso a la Casa de Filtros TG11 - Sensor 01

G1_AT2	:	Temperatura de Ingreso a la Casa de Filtros TG11 - Sensor 02

G1_AT3	:	Temperatura de Ingreso a la Casa de Filtros TG11 - Sensor 03

G2_AT1	:	Temperatura de Ingreso a la Casa de Filtros TG12 - Sensor 01

G2_AT2	:	Temperatura de Ingreso a la Casa de Filtros TG12 - Sensor 02

G2_AT3	:	Temperatura de Ingreso a la Casa de Filtros TG12 - Sensor 03

G1_FQG	:	Flujo de combustible para combustión TG11

G2_FQG	:	Flujo de combustible para combustión TG12

G1_CSBHX:	Apertura de las IBH TG11

G2_CSBHX:	Apertura de las IBH TG12

PR_SR	:	Set Point RPF

STSURS_SR:	STATUS

BP_SR	:	Punto Base

STSAGC_SR:	ESTADO AGC

LLRA_SR	:	LIMITE INFERIOR

HLRA_SR	:	LIMITE SUPERIOR

LR_SR	:	SET LOCAL/REMOTO RSF

FR_SR	:	ESTADO DE SEGUIMIENTO RSF

STATUS_RSF:	STATUS_RSF

11EIC3920:	Set Potencia TG11

12EIC3920:	Set Potencia TG12





### Carga de Datos

In [68]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/03. Machine Learning - Coding Dojo/00. Actividades Plataforma/01. PROYECTO 02/Extractor_Variables_Compresor_Version_02.xlsx',
                   skiprows=2)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3745 entries, 0 to 3744
Data columns (total 34 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   DATE         3745 non-null   datetime64[ns]
 1   G1_DWATT     3745 non-null   float64       
 2   G2_DWATT     3745 non-null   float64       
 3   S1_DWATT     3745 non-null   float64       
 4   G1_CTIM      3745 non-null   float64       
 5   G2_CTIM      3745 non-null   float64       
 6   G1_CPR       3745 non-null   float64       
 7   G2_CPR       3745 non-null   float64       
 8   G1_CTD       3745 non-null   float64       
 9   G2_CTD       3745 non-null   float64       
 10  G1_CPD       3745 non-null   float64       
 11  G2_CPD       3745 non-null   float64       
 12  G1_AT1       3745 non-null   float64       
 13  G1_AT2       3745 non-null   float64       
 14  G1_AT3       3745 non-null   float64       
 15  G2_AT1       3745 non-null   float64       
 16  G2_AT2

,DATE,G1_DWATT,G2_DWATT,S1_DWATT,G1_CTIM,G2_CTIM,G1_CPR,G2_CPR,G1_CTD,G2_CTD,...,BP_SR,STSAGC_SR,LLRA_SR,HLRA_SR,LR_SR,FR_SR,STATUS_RSF,11EIC3920,12EIC3920,Unnamed: 33
0,2023-07-07 00:00:00,182.464020,182.815323,185.923462,50.658714,50.662487,16.292986,15.948944,721.421936,709.082214,...,535.656738,1,540.774353,540.089722,0,1.159668,0,182.629898,182.632828,NaN
1,2023-07-07 00:15:00,178.848755,178.347397,184.303558,50.256260,50.407425,15.921415,15.638843,710.943787,701.566467,...,535.656677,1,540.765564,540.109680,0,1.159668,0,178.586090,178.150406,NaN
2,2023-07-07 00:30:00,185.135635,184.782959,186.302948,50.537636,50.722748,16.467556,16.128410,724.413208,712.161804,...,535.656677,1,540.748047,540.101501,0,1.159665,0,185.032745,184.888687,NaN
3,2023-07-07 00:45:00,183.619492,184.081924,186.112854,50.449390,50.741253,16.328714,16.006746,720.817444,709.464478,...,535.647278,1,540.747070,540.090515,0,1.159668,0,183.977829,184.566422,NaN
4,2023-07-07 01:00:00,185.168991,184.731506,184.871857,50.201321,50.684986,16.455938,16.089300,723.259033,711.226379,...,535.649536,1,540.746582,540.106262,0,1.159763,0,185.032745,184.859390,NaN


In [69]:
df = df.drop(columns=['Unnamed: 33'])

In [70]:
df['DATE'] = pd.to_datetime(df['DATE'])

### Filtrado de datos

In [71]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
DATE,3745,2023-07-26 11:59:59.999999744,2023-07-07 00:00:00,2023-07-16 18:00:00,2023-07-26 12:00:00,2023-08-05 06:00:00,2023-08-15 00:00:00,NaN
G1_DWATT,3745.0,162.974933,-0.5509,181.9846,185.3056,188.472672,191.096512,57.107194
G2_DWATT,3745.0,166.132526,-1.7405,182.173157,185.339584,189.0299,191.242065,51.128144
S1_DWATT,3745.0,166.882725,-360.0039,185.0254,186.516266,190.177063,192.6746,51.273771
G1_CTIM,3745.0,54.440045,0.0,49.7071,50.1816,50.820312,102.2714,10.595297
G2_CTIM,3745.0,55.088502,0.0,50.0088,50.552704,51.237373,96.7304,11.305106
G1_CPR,3745.0,14.655825,0.0,16.214502,16.4762,16.550365,16.6556,4.560582
G2_CPR,3745.0,14.760959,0.0,15.8962,16.1333,16.4288,16.5179,3.879876
G1_CTD,3745.0,661.866258,0.0,716.711365,723.337,725.350403,758.329,180.224995
G2_CTD,3745.0,676.971912,0.0,706.3065,711.545166,720.054,753.2491,128.236568


Antes de separar unidades con sus respectivas variables, necesitamos identificar en qué tipo de estado realizaremos el cálculo de la eficiencia del compresor y de la descripción del ensayo observamos que en las potencias de las unidades se presentan potencias desde 0 a 185. Lo que nos indica que se presentaros parada de planta de la unidad, es decir.

Los criterios que utilizaremos para realizar el filtro de condiciones operativas de la Central de Ciclo combinado serán :



*   Operación de la planta en carga base, es decir:
 *   Potencia TG11, TG12, TV10 mayor a 175 MW
 *   Operación del sistema Chiller, el cual lleva valores de seteo de temperatura de ingreso al compresor a 48°F
 *   Operación de la central sin estar en modo RSF, lo cual hace oscilar la potencia de la planta de acuerdo a la demanda del sistema.
  



*Filtro por potencia de las unidades*

In [72]:
fig = px.line(df, x='DATE', y=['G1_DWATT', 'G2_DWATT', 'S1_DWATT'], title='Potencia de las Unidades de Generación del 07-Jul-2023 al 14-Ago-2023')
fig.show()

*Diagrama de cargas de las unidades de generación de la central termoeléctrica de ciclo combinado TG11 TG12 & TV10*

Por inspección visual observamos que existen valores outliers para la turbina a vapor, ya que esta no puede brindar valores negativos de potencia. Motivo por el cual para eliminar los valores outliers y solo trabajar con potencias base.

In [73]:
TG11_filter = df['G1_DWATT'] > 175
TG12_filter = df['G2_DWATT'] > 175
TV10_filter = df['S1_DWATT'] > 175
df = df.loc[(TG11_filter & TG12_filter & TV10_filter),:]

In [74]:
fig = px.line(df, x='DATE', y=['G1_DWATT', 'G2_DWATT', 'S1_DWATT'], title='Potencia de las Unidades de Generación del 07-Jul-2023 al 14-Ago-2023')
fig.show()

*Diagrama de carga despues de filtrar potencia para asegurar que se esté operado en ciclo combinado 2x1*

Observamos que la data tiene oscilaciones, esta se debe ya que la planta usualmente sube y baja carga dependiendo de la demanda del sistema, necesitamos filtrar mejor la data para solo quedarnos en una condición operativa, la cual será en carga base, con chiller en servicio(el cual hace ganar una potencia de 12MW por tubina a gas) y sin dar RSF al sistema.

In [75]:
fig = px.line(df, x='DATE', y=['G1_CTIM', 'G2_CTIM'], title='Temperatura de Ingreso al compresor del 07-Jul-2023 al 14-Ago-2023')
fig.show()

*Temperatura de ingreso al compresor, para validación de chiller en servicio.*

In [76]:
CTIM_filter = df['G1_CTIM'] < 52 # Esto asegura que el sistema Chiller esté en servicio, ya que este lleva la temperatura de ingreso del compresor a temperaturas de diseño
df = df.loc[(CTIM_filter),:]

In [77]:
fig = px.line(df, x='DATE', y=['G1_CTIM', 'G2_CTIM'], title='Temperatura de Ingreso al compresor del 07-Jul-2023 al 14-Ago-2023')
fig.show()

para poder filtrar que la central no esté brindando RSF

* PR_SR	:	Set Point RPF

* STSURS_SR:	STATUS

* BP_SR	:	Punto Base

* STSAGC_SR:	ESTADO AGC

* LLRA_SR	:	LIMITE INFERIOR

* HLRA_SR	:	LIMITE SUPERIOR

* LR_SR	:	SET LOCAL/REMOTO RSF

* FR_SR	:	ESTADO DE SEGUIMIENTO RSF

* STATUS_RSF:	STATUS_RSF

In [78]:
fig = px.line(df, x='DATE', y=['STSURS_SR','LR_SR','STATUS_RSF'], title='Variables indicadoras de estatus de RPF 07-Jul-2023 al 14-Ago-2023')
fig.show()

Para asegurar la operación sin Reserva Secundaria de Frecuencia, utilizaremos todos los valores de seteo

In [79]:
df[['STSURS_SR','LR_SR','STATUS_RSF']].value_counts()

STSURS_SR  LR_SR  STATUS_RSF
0          0      0             2678
1          0      0              336
           1      1                2
Name: count, dtype: int64

In [86]:
SR_filter = df['STSURS_SR'] < 1
LR_SR_filter = df['LR_SR'] < 1
RSF_filter = df['STATUS_RSF'] < 1
df = df.loc[(SR_filter & LR_SR_filter & RSF_filter),:]
df

,DATE,G1_DWATT,G2_DWATT,S1_DWATT,G1_CTIM,G2_CTIM,G1_CPR,G2_CPR,G1_CTD,G2_CTD,...,STSURS_SR,BP_SR,STSAGC_SR,LLRA_SR,HLRA_SR,LR_SR,FR_SR,STATUS_RSF,11EIC3920,12EIC3920
0,2023-07-07 00:00:00,182.464020,182.815323,185.923462,50.658714,50.662487,16.292986,15.948944,721.421936,709.082214,...,0,535.656738,1,540.774353,540.089722,0,1.159668,0,182.629898,182.632828
1,2023-07-07 00:15:00,178.848755,178.347397,184.303558,50.256260,50.407425,15.921415,15.638843,710.943787,701.566467,...,0,535.656677,1,540.765564,540.109680,0,1.159668,0,178.586090,178.150406
2,2023-07-07 00:30:00,185.135635,184.782959,186.302948,50.537636,50.722748,16.467556,16.128410,724.413208,712.161804,...,0,535.656677,1,540.748047,540.101501,0,1.159665,0,185.032745,184.888687
3,2023-07-07 00:45:00,183.619492,184.081924,186.112854,50.449390,50.741253,16.328714,16.006746,720.817444,709.464478,...,0,535.647278,1,540.747070,540.090515,0,1.159668,0,183.977829,184.566422
4,2023-07-07 01:00:00,185.168991,184.731506,184.871857,50.201321,50.684986,16.455938,16.089300,723.259033,711.226379,...,0,535.649536,1,540.746582,540.106262,0,1.159763,0,185.032745,184.859390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3740,2023-08-14 23:00:00,189.805800,189.700900,187.800700,50.519800,50.780200,16.624900,16.486800,726.593100,721.248700,...,0,539.032800,1,511.269400,550.634700,0,2.160600,0,189.574700,189.605500
3741,2023-08-14 23:15:00,184.602000,184.622500,187.591500,50.472800,50.750900,16.499400,16.115500,723.579800,711.049400,...,0,539.025900,1,511.255100,550.651100,0,2.156000,0,185.032700,184.654300
3742,2023-08-14 23:30:00,185.040800,184.807700,186.330500,49.940600,50.932800,16.502800,16.175300,723.379300,712.823900,...,0,539.016700,1,511.267100,550.634800,0,2.160400,0,185.413700,185.240200
3743,2023-08-14 23:45:00,183.693100,183.433200,184.813300,49.270700,50.278800,16.329100,15.991600,717.663900,707.287700,...,0,539.025800,1,511.285400,550.653100,0,2.160600,0,183.421100,183.511700


In [94]:
fig1 = px.line(df, x='DATE', y=['G1_DWATT', 'G2_DWATT', 'S1_DWATT'], title='Potencia de las Unidades de Generación del 07-Jul-2023 al 14-Ago-2023',
              labels={'value': 'Potencia (DWATT)'})
fig2 = px.line(df, x='DATE', y=['G1_CTIM', 'G2_CTIM'], title='Potencia de las Unidades de Generación del 07-Jul-2023 al 14-Ago-2023',
              labels={'value': 'Temperatura Ingreso al Compresor °F'})
fig1.show()
fig2.show()

*Aseguramos que la potencia de las unidades sean mayores a 175, que el sistema chiller esté en funcionamiento y que no se esté orfertando RSF, que haría variar la potencia bruscamente*

Observamos que hay variaciones de potencia, lo que puede significar que la carga ha estado oscilando por el RPF. Podemso deducir que los picos más altos son cuando se opera a plena carga y sin RPF lo que significaría que la unidad está liberada.

Se tratará de clasificar para poder trabajar la data